In [6]:
import os
import uproot
import ROOT as r
import math
import awkward as ak
import hist
from hist import Hist
from hist import loc
import sys
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
sys.path.append('/sdf/group/hps/users/alspellm/projects/THESIS/analysis/reach_estimate')
from SimpEquations_validated import SimpEquations as simpeqs
import mpl_plot_utilities as mplutils
from matplotlib.backends.backend_pdf import PdfPages
import psutil
import glob
import re

In [8]:
outdir = 'results_summary'
outfilename = 'deltaZ_opt_results'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
def sort_key(file_path):
    # Extract the number following 'mass' using regular expressions
    match = re.search(r"mass_([\d.]+)", file_path)
    if match:
        # Convert the matched number to float for proper sorting
        return float(match.group(1))
    else:
        # If 'mass' number not found, return a large number to push it to the end
        return float('inf')
    
directory = '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2'
pattern = '*.root'
file_list = sorted(glob.glob(f"{directory}/{pattern}"), key=sort_key)
print(file_list)

outfile = r.TFile(f'{outdir}/{outfilename}.root', "RECREATE")

masses = []
best_nsigs = []
best_zbis = []
best_cutvals = []
best_zcuts = []
best_nbkgs = []

best_nsigs_right = []
best_zbis_right = []
best_cutvals_right = []
best_zcuts_right = []
best_nbkgs_right = []

best_nsigs_left = []
best_zbis_left = []
best_cutvals_left = []
best_zcuts_left = []
best_nbkgs_left = []

for filepath in file_list:
    filename = os.path.basename(filepath)
    print(filename)
    mass = filename.split('_')[1]
    
    with uproot.open(f'{directory}/{filename}') as file:
        keys = file.keys()
        if len(keys) == 0:
            break
        masses.append(float(mass))
        print('Mass:', mass)
        nsig_g = file['nsig_g'].values()[1]
        zbi_g = file['zbis_g'].values()[1]
        cutval_g = file['cut_values_g'].values()[1]
        zcut_g = file['zcuts_g'].values()[1]
        nbkg_g = file['nbkg_g'].values()[1]
        fit_probs = file['bkg_fit_p_g'].values()[1]
        
        #Filter out bad background models
        bad_fit_idxs = np.where(fit_probs < 0.05)
        zbi_g = np.delete(zbi_g, bad_fit_idxs)
        nsig_g = np.delete(nsig_g, bad_fit_idxs)
        nbkg_g = np.delete(nbkg_g, bad_fit_idxs)
        zcut_g = np.delete(zcut_g, bad_fit_idxs)
        cutval_g = np.delete(cutval_g, bad_fit_idxs)
        
        
        #Get Best ZBi
        best_zbi_idx = np.argmax(zbi_g)
        best_nsig = nsig_g[best_zbi_idx]
        best_nbkg = nbkg_g[best_zbi_idx]
        best_zcut = zcut_g[best_zbi_idx]
        best_zbi = zbi_g[best_zbi_idx]
        best_cutval = cutval_g[best_zbi_idx]
        
        best_nsigs.append(best_nsig)
        best_nbkgs.append(best_nbkg)
        best_zbis.append(best_zbi)
        best_zcuts.append(best_zcut)
        best_cutvals.append(best_cutval)
        
        '''
        def find_closest_index_left(arr, max_index, target_value):
            left_of_max = arr[:max_index]
            # Find the index where the value is closest to the target value
            index = np.abs(left_of_max - target_value).argmin()
            return index
        
        def find_closest_index_right(arr, max_index, target_value):
            right_of_max = arr[max_index:]
            # Find the index where the value is closest to the target value
            index = np.abs(right_of_max - target_value).argmin()
            return index
        '''
        
        '''
        #Find last iteration where ZBi > 0.9*Best ZBi
        zbi_90p_idxs = np.where(abs(zbi_g) > abs(0.9*best_zbi))[0]
        print(zbi_90p_idxs)
        right_index = zbi_90p_idxs[-1] if len(zbi_90p_idxs) > 0 else None
        left_index = zbi_90p_idxs[0] if len(zbi_90p_idxs) > 0 else None
        '''
        #Find last iteration where ZBi > 0.9*Best ZBi
        print('peak zbi idx:', best_zbi_idx)
        #zbi_left = zbi_g[:best_zbi_idx]
        #zbi_left_idx = np.where(abs(zbi_left) > abs(0.9*best_zbi))[0]
        
        zbi_90p_idxs = np.where(abs(zbi_g) > abs(0.9*best_zbi))[0]
        print(zbi_90p_idxs)
        #print('left:', zbi_90p_idxs[:int(np.where(zbi_90p_idxs == best_zbi_idx)[0])])
        left_index = zbi_90p_idxs[:int(np.where(zbi_90p_idxs == best_zbi_idx)[0])][-1]
        right_index = zbi_90p_idxs[int(np.where(zbi_90p_idxs == best_zbi_idx)[0])+1:][0]
        print('left index', left_index)
        print('right index:', right_index)
        
        #last index above 
        best_nsigs_right.append(nsig_g[right_index])
        best_zbis_right.append(zbi_g[right_index])
        best_cutvals_right.append(cutval_g[right_index])
        best_zcuts_right.append(zcut_g[right_index])
        best_nbkgs_right.append(nbkg_g[right_index])
        
        #first index above
        best_nsigs_left.append(nsig_g[left_index])
        best_zbis_left.append(zbi_g[left_index])
        best_cutvals_left.append(cutval_g[left_index])
        best_zcuts_left.append(zcut_g[left_index])
        best_nbkgs_left.append(nbkg_g[left_index])
       

['/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_40.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_45.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_50.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_55.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_60.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_65.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_70.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_75.0_deltaz_opt.root', '/sdf/g

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
 
outfile.cd()

#Nsig
c = r.TCanvas('expected_signal_deltaZ','expected_signal_deltaZ', 2400, 1400)
c.cd()
nsig_g = r.TGraph(len(masses), np.array(masses), np.array(best_nsigs))
nsig_g.SetName('expected_signal_g')
nsig_g.SetTitle('Expected Signal deltaZ Max ZBi; VD Mass [GeV]; Tenpct Expected Signalx5')
nsig_g.Draw()
nsig_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#ZBi
c = r.TCanvas('max_ZBi_deltaZ','max_ZBi_deltaZ', 2400, 1400)
c.cd()
zbi_g = r.TGraph(len(masses), np.array(masses), np.array(best_zbis))
zbi_g.SetName('max_ZBi_g')
zbi_g.SetTitle('deltaZ Max ZBi; VD Mass [GeV]; Tennpct Signalx5 Max ZBi')
zbi_g.Draw()
zbi_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#nbkgs
c = r.TCanvas('nbkg_deltaZ','nbkg_deltaZ', 2400, 1400)
c.cd()
nbkg_g = r.TGraph(len(masses), np.array(masses), np.array(best_nbkgs))
nbkg_g.SetName('expected_signal_g')
nbkg_g.SetTitle('Nbkg deltaZ Max ZBi; VD Mass [GeV]; Nbkg')
nbkg_g.Draw()
nbkg_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#Zcut
c = r.TCanvas('zcut_nbkglt0pt5_deltaZ','zcut_nbkglt0pt5_deltaZ', 2400, 1400)
c.cd()
zbi_g = r.TGraph(len(masses), np.array(masses), np.array(best_zcuts))
zbi_g.SetName('zcut_nbkglt0[t5_deltaZ_g')
zbi_g.SetTitle('Zcut nbkg lt 0pt5 deltaZ Max ZBi; VD Mass [GeV]; Zcut [mm]')
zbi_g.Draw()
zbi_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#cutvalues
c = r.TCanvas('deltaZ_cut','deltaZ_cut', 2400, 1400)
c.cd()
cutval_g = r.TGraph(len(masses), np.array(masses), np.array(best_cutvals))
cutval_g.SetName('deltaZ_cut_g')
cutval_g.SetTitle('deltaZ Cut Value Max ZBi; VD Mass [GeV]; DeltaZ Cut')
cutval_g.Draw()
cutval_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')
  

In [ ]:
  
    
#TgraphErros with 0.9*ZbiMax Interval
#Nsig
xerrors_l = np.zeros(len(masses))
xerrors_h = np.zeros(len(masses))

c = r.TCanvas('deltaZ_cut_values_90p','deltaZ_cut_values_90p', 2400, 1400)
c.cd()
cutval_hl_g = r.TGraphAsymmErrors(len(masses), np.array(masses), np.array(best_cutvals), xerrors_l, xerrors_h, np.array(best_cutvals)-np.array(best_cutvals_right), np.array(best_cutvals_left)-np.array(best_cutvals))
cutval_hl_g.SetName('deltaZ_cut_values_90p_g')
cutval_hl_g.SetTitle('deltaZ Cut Values 90pct Max ZBi; VD Mass [GeV]; deltaZ [mm]')
cutval_hl_g.Draw()
linear_func = r.TF1("linear_func", "[0] + [1]*x", 40.0, 130.0)
cutval_hl_g.Fit(linear_func,"","",40.0,130.0)
cutval_hl_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')